In [1]:
import pandas as pd

In [63]:
# read tab separated txt file
# df = pd.read_csv('CV_data/Predicted_Targets_Info.default_predictions.txt', sep='\t')
df = pd.read_csv('/home/jsw/Downloads/miRDB_v6.0_prediction_result.txt', sep='\t', header = None)
df.columns = ['microRNA', 'target', 'score']
df = df[df['microRNA'].str.contains('hsa-miR')]
df['microRNA'] = df['microRNA'].str.replace('hsa-miR-', 'miR-', regex = False)

In [64]:
df

,microRNA,target,score
1811610,miR-1-3p,NM_001171020,64.219200
1811611,miR-1-3p,NM_182607,78.723900
1811612,miR-1-3p,NM_001017395,83.220339
1811613,miR-1-3p,NM_001204878,78.995200
1811614,miR-1-3p,NM_032013,65.943800
...,...,...,...
5151913,miR-99b-5p,NM_006041,89.138400
5151914,miR-99b-5p,NM_001144832,74.006100
5151915,miR-99b-5p,NM_004958,92.610030
5151916,miR-99b-5p,NM_005808,69.643800


In [69]:
# save df
df.to_csv('miRDB_v6.0_prediction_result.txt', sep='\t', index = False)

In [65]:
mir_to_gene = df.groupby('microRNA')['target'].apply(list).to_dict()

In [66]:
# Load miR cnv data ============================================================

df_gain = pd.read_csv('extracted_genesets/gain_geneset.csv', index_col = 0)

df_gain.index = df_gain.index.str.replace(r'^MIR', 'miR-', regex = True)
df_gain.index = df_gain.index.str.replace(r'^hsa-mir-', 'miR-', regex = True)

df_gain_mir = df_gain[df_gain.index.str.contains('miR-')]

df_gain_mir_unique = df_gain_mir[~df_gain_mir.index.duplicated(keep='first')]
df_gain_mir_unique

,Del_diff,Diploid_diff,Gain_diff
Gene Symbol,,,
miR-106B,0.44,-50.50,50.05
miR-1200,0.77,-56.89,56.12
miR-129-1,-0.09,-47.91,48.01
miR-148A,0.39,-57.42,57.02
miR-153-2,0.98,-50.29,49.29
...,...,...,...
miR-548f-4,0.35,-48.87,48.52
miR-548n,0.59,-57.06,56.47
miR-548o,0.44,-49.06,48.62


In [67]:
# 새로운 DataFrame 생성 (유전자 이름을 index로, 초기값은 0)
genes = list(set(gene for gene_list in mir_to_gene.values() for gene in gene_list))
new_df = pd.DataFrame(0, index=genes, columns=['expression'])
new_df

,expression
NM_080802,0
NM_001261395,0
NM_001319993,0
NM_001135092,0
NM_015063,0
...,...
NM_001302679,0
NM_001168300,0
NM_001282990,0
NM_001001786,0


In [68]:
# df_gain_mir_unique의 각 miR에 대해 해당하는 유전자들의 expression 값을 -1씩 감소
for mir in df_gain_mir_unique.index:
    if mir in mir_to_gene:
        for gene in mir_to_gene[mir]:
            if gene in new_df.index:
                new_df.loc[gene, 'expression'] -= 1
    else:
        print(f'Warning: {mir} not found in mir_to_gene dictionary.')

In [62]:
new_df

,expression
NM_080802,0
NM_001261395,0
NM_001319993,0
NM_001135092,0
NM_015063,0
...,...
NM_001302679,0
NM_001168300,0
NM_001282990,0
NM_001001786,0
